# Sparameters

gdsfactory provides you with a Lumerical FDTD interface to calculate Sparameters

by default another repo [gdslib](https://gdslib.readthedocs.io/en/latest/index.html) stores the Sparameters

You can chain the Sparameters to calculate solve of larger
circuits using a circuit solver such as:

- Lumerical interconnect
- [simphony (open source)](https://simphonyphotonics.readthedocs.io/en/latest/)


If the Sparameters exists in `gdslib` you can access them really fast.

In [ ]:
# NBVAL_SKIP
import gdsfactory as gf
import gdsfactory.sp as sp
import lumapi

s = lumapi.FDTD()

In [ ]:
gf.components.factory.keys()

In [ ]:
components = [
    "bend_euler",
    "bend_s",
    "coupler",
    "coupler_ring",
    "crossing",
    "mmi1x2",
    "mmi2x2",
    "taper",
    "straight",
]
need_review = []

for component_name in components:
    component = gf.components.factory[component_name]()
    sp.write(component, run=False, session=s)
    response = input(f"does the simulation for {component_name} look good? (y/n)")
    if response.upper()[0] == "N":
        need_review.append(componnent_name)

In [ ]:
components = ["crossing", "mmi1x2", "mmi2x2", "taper", "straight"]
need_review = []

for component_name in components:
    component = gf.components.factory[component_name]()
    sp.write(component, run=False, session=s)
    response = input("does the simulation look good? (y/n)")
    if response.upper()[0] == "N":
        need_review.append(component_name)

In [ ]:
sp.plot(gf.components.mmi1x2(), keys=["S23m", "S13m"], logscale=True)

In [ ]:
sp.write(gf.components.mmi1x2(), layer_to_thickness={(1, 0): 210})

In [ ]:
layer_to_thickness = gf.tech.LAYER_STACK.get_layer_to_thickness()
component = gf.c.bend_circular(radius=3)
component = component.copy()
component.remove_layers(component.layers - set(layer_to_thickness.keys()))
component._bb_valid = False

c = gf.components.extension.extend_ports(component=component, length=1.0)
c.show()
gdspath = c.write_gds()

c

gdsfactory can also compute the Sparameters of a component that have not been simulated before.

In [ ]:
# NBVAL_SKIP

cs = [
    gf.components.coupler_ring(gap=gap, radius=radius)
    for gap in [0.15, 0.2, 0.3]
    for radius in [5, 10]
]

for c in cs:
    c.show()
    print(c)
    sp.write(c)

To debug a simulation you can create a Lumerical session outside the simulator, pass it to the simulator, and use `run=False` flag

In [ ]:
# NBVAL_SKIP
import lumapi

s = lumapi.FDTD()
c = gf.components.straight()
sp.write(c, run=False, session=s)

By default gdsfactory uses the generic LayerStack for 220nm height silicon layer.

You can also define your a different LayerStack

In [ ]:
import gdsfactory as gf

def get_layer_stack():
    return gf.tech.LayerStack(levels = [gf.tech.LayerLevel(name='WG', layer=(2, 0), thickness=400.0, zmin=0.0, material="sin")])


ls = get_layer_stack()